In [7]:
import pandas as pd
import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras.layers import (
    Input, Embedding, Flatten, Dot, Add, Dense, Dropout, Concatenate
)
from tensorflow.keras.regularizers import l2
from tensorflow.keras.utils import plot_model
from sklearn.preprocessing import LabelEncoder
from sample import get_saved_samples
from tensorflow.keras.callbacks import EarlyStopping

In [8]:
# Assuming 'getSavedSamples()' is defined elsewhere and available
users, ratings, books = get_saved_samples()

In [9]:
nUsers = ratings['User-ID'].nunique()
nBooks = ratings['ISBN'].nunique()

# Embedding size and regularization parameter
embeddingSize = 15
regularization = 1e-6

In [10]:
# User Embeddings
userInput = Input(shape=(1,))
userEmbedding = Embedding(nUsers, embeddingSize, embeddings_regularizer=l2(regularization), input_length=1)(userInput)
userVector = Flatten()(userEmbedding)
userBias = Embedding(nUsers, 1, input_length=1)(userInput)
userBias = Flatten()(userBias)

# Book Embeddings
bookInput = Input(shape=(1,))
bookEmbedding = Embedding(nBooks, embeddingSize, embeddings_regularizer=l2(regularization), input_length=1)(bookInput)
bookVector = Flatten()(bookEmbedding)
bookBias = Embedding(nBooks, 1, input_length=1)(bookInput)
bookBias = Flatten()(bookBias)

In [11]:
# Concatenate user and book vectors
concatenated = Concatenate()([userVector, bookVector])

# Increase the dropout rate
dropoutRate = 0.5
dense1 = Dense(64, activation='relu')(concatenated)
dropout1 = Dropout(dropoutRate)(dense1)
dense2 = Dense(32, activation='relu')(dropout1)
dropout2 = Dropout(dropoutRate)(dense2)
output = Dense(1)(dropout2)

# Add biases
output = Add()([output, userBias, bookBias])

# Construct the model
model = Model(inputs=[userInput, bookInput], outputs=output)
model.compile(optimizer='adam', loss='mean_squared_error')

In [12]:
# %%
# Preparing the dataset for training
userIdEncoder = LabelEncoder()
userIds = userIdEncoder.fit_transform(ratings['User-ID'].astype(str))

bookIdEncoder = LabelEncoder()
bookIds = bookIdEncoder.fit_transform(ratings['ISBN'].astype(str))
ratings = ratings['Book-Rating'].values

# Early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Train the model
model.fit([userIds, bookIds], ratings, epochs=10, validation_split=0.2)

Epoch 1/10


2695/2695 [==============================] - 46s 17ms/step - loss: 12.9034 - val_loss: 15.6324
Epoch 2/10
2695/2695 [==============================] - 43s 16ms/step - loss: 9.2620 - val_loss: 17.2307
Epoch 3/10
2695/2695 [==============================] - 42s 16ms/step - loss: 6.4942 - val_loss: 19.1634
Epoch 4/10
2695/2695 [==============================] - 40s 15ms/step - loss: 5.3967 - val_loss: 19.6947
Epoch 5/10
2695/2695 [==============================] - 43s 16ms/step - loss: 4.8372 - val_loss: 19.9897
Epoch 6/10
2695/2695 [==============================] - 43s 16ms/step - loss: 4.4637 - val_loss: 18.4111
Epoch 7/10
2695/2695 [==============================] - 46s 17ms/step - loss: 4.1663 - val_loss: 18.4765
Epoch 8/10
2695/2695 [==============================] - 45s 17ms/step - loss: 3.9237 - val_loss: 18.0707
Epoch 9/10
2695/2695 [==============================] - 42s 16ms/step - loss: 3.7058 - val_loss: 18.0232
Epoch 10/10
2695/2695 [==============================] - 45s 17ms